# Managed Spot Training for XGBoost

---

## Normal Conf

In [3]:
import sys
!{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=1.71.0,<2.0.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.140 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [7]:
%%time

import io
import os
import boto3
import sagemaker
import urllib

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sess = sagemaker.Session()

bucket = "test-sagemaker-examples-1357942113492"
prefix = "DEMO-Spot"

CPU times: user 317 ms, sys: 32 ms, total: 349 ms
Wall time: 646 ms


In [8]:
%%time

FILE_DATA = "abalone"
urllib.request.urlretrieve("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/abalone", FILE_DATA)
sess.upload_data(FILE_DATA, bucket=bucket, key_prefix=prefix + '/train')

CPU times: user 60.8 ms, sys: 12.3 ms, total: 73.2 ms
Wall time: 1.95 s


's3://test-sagemaker-examples-1357942113492/DEMO-Spot/train/abalone'

In [9]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, "xgboost", "1.0-1")

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [10]:
hyperparam = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "silent":"0",
    "objective":"reg:squarederror",
    "num_round":"50"
}

instance_type = "ml.m5.2xlarge"
output_path = f's3://{bucket}/{prefix}/abalone-xgb/output'
content_type = "libsvm"

In [14]:
import time
from time import gmtime, strftime

job_name = f"jumpstart-example-xgboost-spot-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

print("Training job", job_name)

train_use_spot = True
train_max_run = 3600
train_max_wait = 7200 if train_use_spot else None
checkpoint_s3_uri = (f's3://{bucket}/{prefix}/checkpoints/{job_name}' if train_use_spot else none)

print("Checkpoint path:", checkpoint_s3_uri)

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    hyperparameters=hyperparam,
    train_instance_count=1,
    train_instance_type=instance_type,
    train_volume_size=5,
    output_path=output_path,
    sagemaker_session=sess,
    train_use_spot_instances=train_use_spot,
    train_max_run=train_max_run,
    train_max_wait=train_max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri
)

train_input = sagemaker.s3_input(
    s3_data=f's3://{bucket}/{prefix}/train', 
    content_type='libsvm')
estimator.fit({'train': train_input}, job_name=job_name)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training job jumpstart-example-xgboost-spot-2023-05-25-05-56-55
Checkpoint path: s3://test-sagemaker-examples-1357942113492/DEMO-Spot/checkpoints/jumpstart-example-xgboost-spot-2023-05-25-05-56-55
2023-05-25 05:56:55 Starting - Starting the training job...
2023-05-25 05:57:10 Starting - Preparing the instances for training......
2023-05-25 05:58:23 Downloading - Downloading input data
2023-05-25 05:58:23 Training - Downloading the training image...
2023-05-25 05:58:43 Training - Training image download completed. Training in progress...[2023-05-25 05:59:04.221 ip-10-0-94-58.eu-west-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoo